In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import matplotlib.pyplot as plt
from wordsegment import load, segment
load()

In [2]:
# read all files
category_df = pd.read_csv('categories.csv')
full_df = pd.read_csv('full_data.csv')
outfit_df = pd.read_csv('outfits.csv')
tag_df = pd.read_csv('tagged_product_attributes.csv')
review_df = pd.read_csv('womens_clothing_reviews.csv')

In [3]:
full_df

,product_id,brand,mpn,product_full_name,description,brand_category,created_at,updated_at,deleted_at,brand_canonical_url,details,labels,bc_product_id
0,01DSE9TC2DQXDG6GWKW9NMJ416,Banana Republic,514683,Ankle-Strap Pump,"A modern pump, in a rounded silhouette with an...",Unknown,2019-11-11 22:37:15.719107+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,"A modern pump, in a rounded silhouette with an...","{""Needs Review""}",NaN
1,01DSE9SKM19XNA6SJP36JZC065,Banana Republic,526676,Petite Tie-Neck Top,Dress it down with jeans and sneakers or dress...,Unknown,2019-11-11 22:36:50.682513+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,Dress it down with jeans and sneakers or dress...,"{""Needs Review""}",NaN
2,01DSJX8GD4DSAP76SPR85HRCMN,Loewe,4.001E+11,52MM Padded Leather Round Sunglasses,Padded leather covers classic round sunglasses.,JewelryAccessories/SunglassesReaders/RoundOval...,2019-11-13 17:33:59.581661+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/loewe-52mm-pad...,100% UV protection\nCase and cleaning cloth in...,"{""Needs Review""}",NaN
3,01DSJVKJNS6F4KQ1QM6YYK9AW2,Converse,4.00012E+11,Baby's & Little Kid's All-Star Two-Tone Mid-To...,The iconic mid-top design gets an added dose o...,"JustKids/Shoes/Baby024Months/BabyGirl,JustKids...",2019-11-13 17:05:05.203733+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/converse-babys...,Canvas upper\nRound toe\nLace-up vamp\nSmartFO...,"{""Needs Review""}",NaN
4,01DSK15ZD4D5A0QXA8NSD25YXE,Alexander McQueen,4.00011E+11,64MM Rimless Sunglasses,Hexagonal shades offer a rimless view with int...,JewelryAccessories/SunglassesReaders/RoundOval,2019-11-13 18:42:30.941321+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/alexander-mcqu...,100% UV protection\nGradient lenses\nAdjustabl...,"{""Needs Review""}",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48974,01DSNVXY8EJ9FQAJ3MPDMPASHD,Bonpoint,4.00091E+11,Baby's Hooded Jacket,NaN,JustKids/Baby024months/InfantGirls/Outerwear,2019-11-14 21:08:28.040417+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/bonpoint-babys...,Cozy double breasted jacket crafted from cotto...,"{""Needs Review""}",NaN
48975,01DSGYHA3RMCHENBJVQPBGXM97,Laura Mercier,4.00096E+11,Flawless Fusion Ultra-Longwear Foundation,"WHAT IT ISA 15-hour long wearing, water resist...",SaksBeautyPlace/ForHer/Color/Foundation/Liquid...,2019-11-12 23:17:47.761072+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/laura-mercier-...,"WHAT IT ISA 15-hour long wearing, water resist...","{""Needs Review""}",NaN
48976,01DSJT8H12CAFQQH07SQSQWJ8C,Splendid,4.001E+11,Baby Girl's 2-Piece Ruffle Sweatshirt & Stripe...,Ruffled-trim sweatshirt lends romance to this ...,"JustKids/Baby024months/InfantGirls/Tops,JustKi...",2019-11-13 16:41:34.491443+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/splendid-baby-...,"Crewneck\nLong sleeves\nRib-knit neck, cuffs a...","{""Needs Review""}",NaN
48977,01DSH2PF9J7QZ44D842B3GMCFN,Florence Eiseman,4.00012E+11,Little Girl's Plaid & Velvet Dress,Pretty plaid dress with velvet collar and velv...,"JustKids/Girls214/ToddlerGirls24/Dresses,JustK...",2019-11-13 00:30:31.212215+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/florence-eisem...,Peter Pan collar\nShort sleeves\nBack zipper\n...,"{""Needs Review""}",NaN


In [4]:
# have to do some cleaning/preprocessing
# since the attribute names and values are not completely identical 
# between the initial_tags and additional tags
tag_df

,product_id,product_color_id,attribute_name,attribute_value,file
0,01DVBTBPHR8WJTCVEN5AJRHF47,01DVBTBPJ41VVT00JJCG8TTZ2W,gender,Women,initial_tags
1,01DVA7QRXM928ZM0WWR7HFNTC1,01DVA7QRXXR9F0TWVE1HMC5ZQ3,Primary Color,Blacks,initial_tags
2,01DPGV4YRP3Z8J85DASGZ1Y99W,01DPGVGBK6YGNYGNF2S6FSH02T,style,Casual,initial_tags
3,01E1JM43NQ3H17PB22EV3074NX,01E1JM5WFWWCCCH3JTTTCYQCEQ,style,Modern,initial_tags
4,01DSE8Z2ZDAZKZ2SKCS1E3B3HK,01DSE8ZG8Y3FR8KWE2TY1QDWBF,shoe_width,Medium,initial_tags
...,...,...,...,...,...
119340,01DT2D3BN9K4SEH71MPB7WD5WA,01DT2D3BNFZBEA1BTZHSGCQN46,fit,relaxed,additional
119341,01DT2D3BN9K4SEH71MPB7WD5WA,01DT2D3BNFZBEA1BTZHSGCQN46,lengthblazerscoatsandjackets,midlengthathips,additional
119342,01DT2D3BN9K4SEH71MPB7WD5WA,01DT2D3BNFZBEA1BTZHSGCQN46,closureblazerscoatsandjackets,buttonedfront,additional
119343,01DT2D3BN9K4SEH71MPB7WD5WA,01DT2D3BNFZBEA1BTZHSGCQN46,occasion,work,additional


In [5]:
def seg(s):
    if not re.search(r'_', s):
        l = segment(s)
        if len(l) == 1:
            return l[0]
        else:
            return '_'.join(l)
    else:
        return s

tag_df['attribute_name_new'] = tag_df.attribute_name.apply(seg)
tag_df['attribute_value_new'] = tag_df.attribute_value.apply(seg)

In [6]:
sorted(tag_df.attribute_name_new.unique())

['additional_color',
 'belt_buckle_material',
 'belt_buckle_shape',
 'belt_closure',
 'belt_material',
 'belt_width',
 'calf_width',
 'category',
 'class_belts',
 'class_blazers_coats_and_jackets',
 'class_booties',
 'class_boots',
 'class_dress',
 'class_flats',
 'class_handbags',
 'class_jumpsuit_and_romper',
 'class_mules_and_slides',
 'class_pants_and_leggings',
 'class_pumps_and_heels',
 'class_sandals',
 'class_shorts',
 'class_skirts',
 'class_slippers',
 'class_sneakers_and_athletic',
 'class_sunglasses',
 'class_wedges',
 'closure_blazers_coats_and_jackets',
 'closure_handbag',
 'closure_one_piece',
 'closure_pants_and_leggings',
 'closure_shoe',
 'closure_shorts',
 'closure_skirts',
 'closure_sweater',
 'closure_top',
 'color',
 'dry_clean_only',
 'embellishment',
 'fit',
 'gender',
 'heel_height',
 'heel_shape',
 'leg_style',
 'leg_style_jeans',
 'length_blazers',
 'length_blazers_coats_and_jackets',
 'length_coats_and_jackets',
 'length_jeans',
 'length_one_piece',
 'length

In [7]:
sorted(tag_df.attribute_value_new.unique())

['100_cashmere',
 '100_cotton',
 '100linen',
 '100silk',
 '100wool',
 '14zip',
 '5pocketpantnondenim',
 'a_line',
 'abstract',
 'accessory',
 'acetate',
 'acid',
 'acrylic',
 'active',
 'active_track',
 'adjustable',
 'alpaca',
 'androgynous',
 'animal',
 'ankle',
 'ankle_strap',
 'anorak_raincoat_windbreaker',
 'asymmetrical',
 'athleisure',
 'aviator_square',
 'aviator_teardrop',
 'back_tie',
 'back_zip',
 'back_zipper',
 'backless',
 'backpacks',
 'baggy',
 'ballet',
 'band_collar',
 'beach_bags',
 'beaded',
 'beiges',
 'belt',
 'belt_bags_fannypack',
 'belted',
 'belts',
 'bermuda',
 'bike',
 'black',
 'blacks',
 'blazer',
 'blazer_dress',
 'blazers_coats_jackets',
 'block',
 'blouse',
 'blues',
 'boatneck',
 'body_con',
 'bodysuit',
 'boho',
 'bomber_varsity',
 'bootcut',
 'bootie',
 'booties',
 'boots',
 'bottom',
 'boyfriend',
 'brass',
 'browns',
 'bucket_bags',
 'buckle',
 'buckles',
 'burgundies',
 'business_casual',
 'business_casual_dress',
 'bustier',
 'button_down',
 'but

In [8]:
clean_tag_df = pd.DataFrame(tag_df[['product_id','attribute_name_new','attribute_value_new']]
                            .groupby(['attribute_name_new','product_id'])
                            .attribute_value_new
                            .apply(set)).reset_index().rename(columns={'attribute_name_new': 'name', 'attribute_value_new': 'value'})
clean_tag_df

,name,product_id,value
0,additional_color,01DPCDEF6SYX2E1NT5X7HJBFGY,{pinks}
1,additional_color,01DPCG1C1P0MQAV9NMS3N1TDAA,"{blacks, pinks, greens}"
2,additional_color,01DPCQ0CNKKHD3899ZEY9SEDHA,{whites}
3,additional_color,01DPCYE6AKJFKJSNCBDTQFG52Y,"{light_brown, grays, whites}"
4,additional_color,01DPD2FGA0FWFXJCKMT5QBJYHK,"{blacks, blues, greens, pinks, whites}"
...,...,...,...
56516,wash,01E606C7SRDEHGS8XZT4ZCNN6Y,{dark_blue}
56517,wash,01E606HQS50FZF40JZD9GDKRQ5,{black}
56518,wash,01E6070892T9HB6NZ6APHTZ70N,{black}
56519,wash,01E60710RJME1ADG2P26BGBQJ2,{dark_blue}


# Decide which N – 2 categories your group would like to focus on. 

You are required to focus on the proprietary attributes – style and occasion. Beyond that, your group should pick N - 2 other groups to analyze.

Examples of categories:
- Embellishments
- Category
- Prints
- Material

In [9]:
Counter(clean_tag_df.name).most_common(10)

[('category', 3969),
 ('primary_color', 3930),
 ('style', 3916),
 ('occasion', 3914),
 ('gender', 3913),
 ('fit', 2949),
 ('sizing', 2941),
 ('material_clothing', 2924),
 ('dry_clean_only', 2776),
 ('sleeve_length', 2047)]

In [10]:
attribute_list = ['category','fit','style','occasion']
for i in attribute_list:
    print(i)
    print(Counter(clean_tag_df[clean_tag_df.name==i].value.apply(tuple)).most_common(),'\n')

category
[(('top',), 963), (('bottom',), 899), (('shoe',), 672), (('one_piece',), 427), (('accessory',), 310), (('sweater',), 293), (('blazers_coats_jackets',), 280), (('sweatshirt_hoodie',), 118), (('accessory', 'top'), 3), (('bottom', 'top'), 1), (('blazers_coats_jackets', 'top'), 1), (('blazers_coats_jackets', 'sweater'), 1), (('top', 'sweatshirt_hoodie'), 1)] 

fit
[(('relaxed',), 1093), (('semi_fitted',), 654), (('straight_regular',), 553), (('fitted_tailored',), 455), (('oversized',), 105), (('semi_fitted', 'relaxed'), 34), (('relaxed', 'straight_regular'), 20), (('relaxed', 'oversized'), 7), (('semi_fitted', 'straight_regular'), 7), (('fitted_tailored', 'semi_fitted'), 6), (('oversized', 'relaxed'), 5), (('semi_fitted', 'fitted_tailored'), 4), (('semi_fitted', 'oversized'), 3), (('relaxed', 'semi_fitted', 'straight_regular'), 2), (('fitted_tailored', 'relaxed'), 1)] 

style
[(('classic', 'casual'), 353), (('edgy', 'modern', 'casual'), 175), (('classic', 'modern', 'casual'), 165)

# Join the tagged_product_attributes table with full_data and investigate the details, descriptions, and tags used for each category

your goal is to get a sense for the business logic and rules used in tagging a certain product category.

In [11]:
df = pd.merge(full_df, clean_tag_df, on = 'product_id')
df['style'] = df[df.name=='style'].value.apply(list)
df['occasion'] = df[df.name=='occasion'].value.apply(list)
df['fit'] = df[df.name=='fit'].value.apply(list)
df['category'] = df[df.name=='category'].value.apply(list)
df = df.drop(['mpn','created_at','updated_at','deleted_at','bc_product_id','labels'], axis=1)

def notna(x):
    for i in x:
        if isinstance(i, list):
            return i
    return ''

df = df.groupby('product_id').agg({
    'brand':'first', 'product_full_name':'first', 'description':'first', 
    'brand_category':'first','brand_canonical_url':'first', 'details':'first',
    'style':notna,'occasion':notna, 'fit':notna,'category':notna})
df

,brand,product_full_name,description,brand_category,brand_canonical_url,details,style,occasion,fit,category
product_id,,,,,,,,,,
01DMBRYVA2P5H24WK0HTK4R0A1,Eileen Fisher,Slim Knit Skirt,A nice skirt,Apparel,NaN,NaN,,,,[bottom]
01DMBRYVA2PEPWFTT7RMP5AA1T,Eileen Fisher,Rib Mock Neck Tank,A nice tank,Apparel,NaN,NaN,,,,[blazers_coats_jackets]
01DMBRYVA2Q2ST7MNYR6EEY4TK,Equipment,Chemelle Midi Dress,A nice dress,Apparel,NaN,NaN,,,,[one_piece]
01DMBRYVA2S5T9W793F4CY41HE,kate spade new york,medium margaux leather satchel,A nice bag,Bags,NaN,NaN,,,,[accessory]
01DMBRYVA2ZFDYRYY5TRQZJTBD,Tory Burch,Penelope Mid Cap Toe Pump,A nice shoe,Shoes,NaN,NaN,,,,[shoe]
...,...,...,...,...,...,...,...,...,...,...
01E6076GTCE5P3VH76VWJH4MY9,Sam Edelman,Krissy Espadrille Flat,"A woven espadrille sole extends the chic, summ...",Unknown,https://shop.nordstrom.com/s/sam-edelman-kriss...,Runs large; order 1/2 size down.\nTextile or l...,"[classic, boho, casual]","[weekend, vacation, day_tonight]",,[shoe]
01E6078G3GRATF2C96VKYYWSGD,ALO,High Waist Moto 7/8 Leggings,Pintucked panels create a cool moto aesthetic ...,Unknown,https://shop.nordstrom.com/s/alo-high-waist-mo...,"XS=0-2, S=4-6, M=8-10, L=12-14. Mid rise.\n24 ...","[athleisure, modern, casual]",[workout],[fitted_tailored],[bottom]
01E6079DG58YW9K78D57C6J2Y1,THE GREAT.,The Ravine Cotton Midi Dress,"Puff sleeves, covered rouleau buttons and an o...",Unknown,https://shop.nordstrom.com/s/the-great-the-rav...,"True to size.\n47"" length (size 1/4-6)\nFront ...","[modern, boho, casual]","[weekend, vacation, day_tonight]",[relaxed],[one_piece]


In [12]:
input_cols = df.columns[:-4]
input_df = df[input_cols].apply(lambda x: x.str.lower()).replace(np.nan,'').astype(str).copy()
input_df

,brand,product_full_name,description,brand_category,brand_canonical_url,details
product_id,,,,,,
01DMBRYVA2P5H24WK0HTK4R0A1,eileen fisher,slim knit skirt,a nice skirt,apparel,,
01DMBRYVA2PEPWFTT7RMP5AA1T,eileen fisher,rib mock neck tank,a nice tank,apparel,,
01DMBRYVA2Q2ST7MNYR6EEY4TK,equipment,chemelle midi dress,a nice dress,apparel,,
01DMBRYVA2S5T9W793F4CY41HE,kate spade new york,medium margaux leather satchel,a nice bag,bags,,
01DMBRYVA2ZFDYRYY5TRQZJTBD,tory burch,penelope mid cap toe pump,a nice shoe,shoes,,
...,...,...,...,...,...,...
01E6076GTCE5P3VH76VWJH4MY9,sam edelman,krissy espadrille flat,"a woven espadrille sole extends the chic, summ...",unknown,https://shop.nordstrom.com/s/sam-edelman-kriss...,runs large; order 1/2 size down.\ntextile or l...
01E6078G3GRATF2C96VKYYWSGD,alo,high waist moto 7/8 leggings,pintucked panels create a cool moto aesthetic ...,unknown,https://shop.nordstrom.com/s/alo-high-waist-mo...,"xs=0-2, s=4-6, m=8-10, l=12-14. mid rise.\n24 ..."
01E6079DG58YW9K78D57C6J2Y1,the great.,the ravine cotton midi dress,"puff sleeves, covered rouleau buttons and an o...",unknown,https://shop.nordstrom.com/s/the-great-the-rav...,"true to size.\n47"" length (size 1/4-6)\nfront ..."


In [ ]:
def process_brand(x):
    return re.sub(r'[^a-z]', ' ', x)

def process_text(x):
    

# Build a model that will takes as input:
- product description (if any)
- product name
- product details (if any)
- brand

# And outputs the predicted attributes of this product.

For example, if the category you are using is fit,

INPUT:
- description: Blush linen Button fastenings along front 100% linen; lining: 100% cotton Dry clean Designer color: Shell Imported
- brand: Zimmermann
- brand_category: Clothing / Jumpsuits / Full Length

The actual clothing’s product URL is here.

OUTPUT:
- predicted fit: RELAXED

# style

In [ ]:
style_df = df[df['style']!=np.nan].copy()
style_list = style_df['style'].explode().unique()
for i in style_list:
    style_df[i] = style_df['style'].apply(lambda x: 1 if i in x else 0)

style_df

In [ ]:
input_cols = style_df.columns[:-13]
input_df = style_df[input_cols].apply(lambda x: x.str.lower())
input_df = input_df.astype(str)
input_df

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB 
from sklearn.metrics import confusion_matrix 
  
# creating bag of words model
def dummy(doc):
    return doc
cv = CountVectorizer(tokenizer=dummy,preprocessor=dummy) 
  
X = cv.fit_transform(input_df.brand.apply(lambda x: [x])).toarray()
y = style_df.modern.values
  
X_train, X_test, y_train, y_test = train_test_split( 
           X, y, test_size = 0.25, random_state = 0) 

# fitting naive bayes to the training set 
from sklearn.naive_bayes import GaussianNB 
from sklearn.metrics import confusion_matrix 
  
classifier = GaussianNB(); 
classifier.fit(X_train, y_train) 
  
# predicting test set results 
y_pred = classifier.predict(X_test) 
  
# making the confusion matrix 
cm = confusion_matrix(y_test, y_pred)
accuracy_score(y_test, y_pred)

In [ ]:
style_cols = style_df.columns[8:]
style_cols

In [ ]:
clean_full_df = full_df.drop(['product_id','mpn','created_at','updated_at','deleted_at','bc_product_id','labels'], axis=1)
clean_full_df = clean_full_df.apply(lambda x: x.str.lower()).astype(str)

In [ ]:
cv = CountVectorizer() 

X = cv.fit_transform(input_df.apply(lambda x: ' '.join(x), axis=1)).toarray()
y = style_df.modern.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

classifier = GaussianNB() 
classifier.fit(X_train, y_train)

In [ ]:
cv.transform('a.l.c').toarray()

# predicting on full data
clean_full_df[i] = classifier.predict_proba()

# occasion

In [ ]:
occasion_df = df[df.category=='occasion'].copy()
occasion_list = occasion_df.value.apply(list).explode().unique()
for i in occasion_list:
    occasion_df[i] = occasion_df.value.apply(lambda x: 1 if i in x else 0)

occasion_df

In [ ]:
occasion_df.drop(['category','value'],axis=1).to_csv('occasion_data.csv',index=False)

# fit

In [ ]:
fit_df = df[df.category=='fit'].copy()
fit_list = fit_df.value.apply(list).explode().unique()
for i in fit_list:
    fit_df[i] = fit_df.value.apply(lambda x: 1 if i in x else 0)

fit_df

In [ ]:
fit_df.drop(['category','value'],axis=1).to_csv('fit_data.csv',index=False)

# category

In [ ]:
category_df = df[df.category=='category'].copy()
category_list = category_df.value.apply(list).explode().unique()
for i in category_list:
    category_df[i] = category_df.value.apply(lambda x: 1 if i in x else 0)

category_df

In [ ]:
category_df.drop(['category','value'],axis=1).to_csv('category_data.csv',index=False)